# Model Prototype

### This notebook shows how to create a baseline model pipeline and save it

##### We save the Spark Dataframe as an Iceberg Table. Iceberg is a new open table format backed by Apple, Netflix and Cloudera. 
##### In the context of ML Ops, the most anticipated feature is Time Travel i.e. the ability to reproduce the data and the schema across different versions in time
##### Finally, we create a simple PySpark pipeline and train a classifier with Keras/Tensorflow

* For a more comprehensive demo of Iceberg in CML, please visit the [Spark3 Iceberg CML Github Repository](https://github.com/pdefusco/Spark3_Iceberg_CML)
* For a more detailed introduction to CML Session, Notebooks, and Spark tips and trips please visit the [CML Total Beginner GitHub Repository](https://github.com/pdefusco/CML-Total-Beginner)
* For a more comprehensive example of the Atlas Python client mentioned below, please visit the [Atlas Client Example Notebook in the Data Integration with ML GitHub Repository](https://github.com/pdefusco/Data_Integration_wMachineLearning/blob/main/2_A_Atlas_Client_Example.ipynb)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from sklearn.datasets import make_circles
import tensorflow as tf
import pandas as pd
from helpers.plot_decision_boundary import *

#### The Spark Session is created with the following configurations. If you get an error, ensure your CML Session is using Runtimes and Spark 3.1.

In [2]:
spark = SparkSession.builder.master('local[*]')\
  .config("spark.jars.packages","org.apache.iceberg:iceberg-spark3-runtime:0.12.1")\
  .config("spark.sql.extensions","org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")\
  .config("spark.sql.catalog.spark_catalog","org.apache.iceberg.spark.SparkSessionCatalog")\
  .config("spark.sql.catalog.spark_catalog.type","hive")\
  .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
  .config("spark.yarn.access.hadoopFileSystems","s3a://gd01-uat2/")\
  .getOrCreate()

#### Just some fake data...

In [4]:
# Make 1000 examples
n_samples = 1000

# Create circles
X, y = make_circles(n_samples, 
                    noise=0.03, 
                    random_state=42)

circles = pd.DataFrame({"var1":X[:, 0], "var2":X[:, 1], "label":y})
circles.head()

,var1,var2,label
0,0.754246,0.231481,1
1,-0.756159,0.153259,1
2,-0.815392,0.173282,1
3,-0.393731,0.692883,1
4,0.442208,-0.896723,0


#### We can save the DataFrame as an Iceberg Table using Spark

In [5]:
# Creating a Spark Dataframe from the Pandas Dataframe
sparkDF=spark.createDataFrame(circles) 

In [12]:
# Saving the Spark Dataframe as an Iceberg table
spark.sql("CREATE TABLE IF NOT EXISTS ice_cml (var1 int, var2 int, label int) USING iceberg")

sparkDF.write.format("iceberg").mode("overwrite").save("default.ice_cml")

#### The table is automatically tracked by the Data Lake associated with the CML Workspace

#### To check that a new entry for the table has been added to Atlas in the Data Lake, go back to the CDP Homepage and open Data Catalog. 

#### Select the Data Lake (i.e. Cloud Environment) that your worskpace was built in. 

#### Use the Atlas Search bar at the top to browse for the table and click on it

#### Notice Atlas is tracking a lot of interesting Metadata including Table Attributes, Lineage, and a lot More. 

#### The Metadata can even be customized. [This notebook](https://github.com/pdefusco/Data_Integration_wMachineLearning/blob/main/2_A_Atlas_Client_Example.ipynb) shows how you can use the Atlas Python Client to build custom lineage flows.

#### Back to Modeling. We will use Keras and Tensorflow to build this classifier. Our data is in Spark though, so we will use Petastorm to transform the data Tensorflow-readable.

In [13]:
!pip3 install petastorm

     |████████████████████████████████| 283 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 291 kB 92.6 MB/s eta 0:00:01
     |████████████████████████████████| 829 kB 113.5 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 101.0 MB/s eta 0:00:01
     |████████████████████████████████| 44 kB 451 kB/s s eta 0:00:01
     |████████████████████████████████| 25.6 MB 116.3 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 880 kB/s s eta 0:00:01
     |████████████████████████████    | 246.9 MB 96.6 MB/s eta 0:00:011

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 281.3 MB 34 kB/s 
     |████████████████████████████████| 198 kB 110.8 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=32fda17f62ac6e87eaef8a6e235595e4c7024d506fdcc3aaba3a14d39688e0fc
  Stored in directory: /home/cdsw/.cache/pip/wheels/6e/9c/ed/4499c9865ac1002697793e0ae05ba6be33553d098f3347fb94
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=1b9e3b29a52b66939ba7f5befa8ff94dbd56c39dcfbe7b8fc7924fe89509f65e
  Stored in directory: /home/cdsw/.cache/pip/wheels/e8/d9/e5/78436a0a3899d81410aeb45b200153113667f2e250f6882ada
Successfully built future pyspark


In [14]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
import tensorflow.compat.v1 as tf  # pylint: disable=import-error

In [21]:
# specify a cache dir first.

# Set a cache directory for intermediate data.
# The path should be accessible by both Spark workers and driver.
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF,"file:///tmp/petastorm/cache/tf-example")

In [ ]:
#!pip3 install s3fs

In [23]:
# create a converter from `df`
# it will materialize `df` to cache dir.
converter = make_spark_converter(sparkDF)

Converting floating-point columns to float32
The median size 5033 B (< 50 MB) of the parquet files is too small. Total size: 10062 B. Increase the median file size by calling df.repartition(n) or df.coalesce(n), which might help improve the performance. Parquet files: file:///tmp/petastorm/cache/tf-example/20211203233933-appid-local-1638570042715-2c963542-b1cd-4cd6-8bf5-1297cbb97040/part-00000-db00b0de-8eba-40c6-8675-01b28dff976b-c000.parquet, ...


In [ ]:
converter

In [24]:
# Create the model (same as model_7)
model = tf.keras.Sequential([
  tf.keras.layers.Dense(4, activation="relu"), # hidden layer 1, using "relu" for activation (same as tf.keras.activations.relu)
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid") # output layer, using 'sigmoid' for the output
])

In [27]:
# Compile the model
model.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), # increase learning rate from 0.001 to 0.01 for faster learning
                metrics=['accuracy'])

In [28]:
# make a tensorflow dataset from `converter`
with converter.make_tf_dataset() as dataset:
    # the `dataset` is `tf.data.Dataset` object
    # we can train/evaluate model on the `dataset`
    history = model.fit(dataset)
    # when exiting the context, the reader of the dataset will be closed
    
    # Evaluate our model on the test set
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f"Model loss on the test set: {loss}")
    print(f"Model accuracy on the test set: {100*accuracy:.2f}%")


# delete the cached files of the dataframe.
converter.delete()

 727007/Unknown - 626s 859us/step - loss: -1534.6987 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [ ]:


# Visualize with a plot
import matplotlib.pyplot as plt
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.RdYlBu);


# Split data into train and test sets
X_train, y_train = X[:800], y[:800] # 80% of the data for the training set
X_test, y_test = X[800:], y[800:] # 20% of the data for the test set

# Check the shapes of the data
X_train.shape, X_test.shape # 800 examples in the training set, 200 examples in the test set

# Set random seed
tf.random.set_seed(42)

# Create the model (same as model_7)
model = tf.keras.Sequential([
  tf.keras.layers.Dense(4, activation="relu"), # hidden layer 1, using "relu" for activation (same as tf.keras.activations.relu)
  tf.keras.layers.Dense(4, activation="relu"),
  tf.keras.layers.Dense(1, activation="sigmoid") # output layer, using 'sigmoid' for the output
])

# Compile the model
model.compile(loss=tf.keras.losses.binary_crossentropy,
                optimizer=tf.keras.optimizers.Adam(lr=0.01), # increase learning rate from 0.001 to 0.01 for faster learning
                metrics=['accuracy'])

# Fit the model
history = model.fit(X_train, y_train, epochs=25)

# Evaluate our model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Model loss on the test set: {loss}")
print(f"Model accuracy on the test set: {100*accuracy:.2f}%")

# Plot the decision boundaries for the training and test sets
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.title("Train")
plot_decision_boundary(model, X=X_train, y=y_train)
plt.subplot(1, 2, 2)
plt.title("Test")
plot_decision_boundary(model, X=X_test, y=y_test)
plt.show()

# You can access the information in the history variable using the .history attribute
pd.DataFrame(history.history)

# Plot the loss curves
pd.DataFrame(history.history).plot()
plt.title("Model training curves")


model.save('models/my_model.h5')